# Import essential libraries

In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import ActionChains
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np

In [4]:
options = Options()
options.headless = True 
custom_user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
options = Options()

# Get main page data

In [6]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

driver.get("https://snappfood.ir/service/all/city/rafsanjan?page=0")
WebDriverWait(driver,15)
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')
driver.close()

In [7]:
names=list()
score=list()
review_num=list()
labels = list()
links= list()


restaurants = soup.find_all('div', class_='sc-citwmv iMkliF')

for restaurant in restaurants:
    names.append( restaurant.find('h2',class_='sc-hKgILt VendorCard__VendorTitle-sc-6qaz7-5 jSKOuL bHWLa').text)
    links.append( restaurant.find('a').get('href'))
    score.append(restaurant.find('span',class_='sc-hKgILt jsaCNc').text)
    review_num.append(restaurant.find('p',class_='sc-hKgILt fYlAbO').text)
    labels.append(restaurant.find('h3',class_='sc-hKgILt fYlAbO').text)

# Getting restaurants information pages separately

In [8]:
empty_list=[[]] * len(names)
restaurants={'restaurant':names,'score':score,'review_num':review_num,'label':labels,'address':empty_list.copy(),
             'link':links,'page_info':empty_list.copy() ,'rating':empty_list.copy(),'leat_shap':empty_list.copy(),
             'comment_number':empty_list.copy(),'comment_scores':empty_list.copy()}

menu= pd.DataFrame(columns=['restaurant','category','food','price'])

In [9]:
def dig(counter):
    print(f'try for {names[counter]}')
    options.add_argument(f'user-agent={custom_user_agent}')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    try:
       
        driver.get('https://snappfood.ir'+links[counter])
        wait = WebDriverWait(driver, 10)

        button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'hMDcUg')))
        ActionChains(driver).move_to_element(button).click(button).perform()


        page_source = driver.page_source
        soup= BeautifulSoup(page_source)
        container = soup.find('section', class_='CategorySections__Container-sc-ls8sfa-3 ccHEix')

        for i in container:
            if i.find('p').text != 'پر‌طرفدارها':
                    category= (i.find('p').text)
                    category_foods= i.find_all('h2','sc-hKgILt esHHju')
                    category_prices= i.find_all('span','sc-hKgILt hxREoh')
                    

                    for food, price in zip(category_foods, category_prices): 
                            data={'restaurant':names[counter],'category':category,'food':food.text,'price':price.text}
                            menu.loc[len(menu)] = data


        button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'hUpWbm')))
        ActionChains(driver).move_to_element(button).click(button).perform()

        
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#modal-backdrop')))
        new_content=wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'VendorDetail__ShiftCard-sc-1nr3673-0')))

        

        soup= BeautifulSoup(new_content.get_attribute('innerHTML'))
        
        restaurants['rating'][counter]= soup.find_all('div',class_=['sc-dlfnbm Scores__RateBar-sc-q0gltu-3','fBrhOL', 'dbSYdh'],type=True)
        restaurants['leat_shap'][counter]= soup.find('span',class_='sc-hKgILt hxREoh').text
        restaurants['address'][counter]= soup.find('p',class_='sc-hKgILt cnaQBH').text
        restaurants['comment_number'][counter]= soup.find_all('span',class_='sc-hKgILt jsaCNc')[1].text
        restaurants['comment_scores'][counter]= [i.text for i in soup.find_all('p',class_='sc-hKgILt Item__Rate-sc-1k0s5ty-6 hmsjTi jFBLxL')]
        
        driver.quit()
        print('it was successful !')

    except Exception as e:
        print('An error occurred, retying again ...')
        driver.quit()
        dig(counter)


In [ ]:
for i in range(len(names)):
    dig(i)

# Store collected data

In [102]:
df=pd.DataFrame(restaurants)
df.sample()

,restaurant,score,review_num,label,address,link,page_info,rating,leat_shap,comment_number,comment_scores
3,فست فود زرین مشرق زمین,۴.۱,(۷۱۴ امتیاز),فست فود، سوخاری، نوشیدنی,خیابان امام خمینی، نبش کوچه ۹، فست فود زرین مش...,/restaurant/menu/%D9%81%D8%B3%D8%AA_%D9%81%D9%...,[],"[[], [], [], [], []]","۴۵,۰۰۰ تومان",۵۸,"[۲, ۴, ۵, ۳, ۴, ۵, ۵, ۳, ۵]"


In [103]:
menu.sample(1)

,restaurant,category,food,price
570,فست فود کنج گلستان,برگر,ماشروم برگر,"۱۹۵,۰۰۰ تومان"


In [ ]:
df.to_csv('./raw data.csv')
menu.to_csv('./menu.csv')